In [9]:
import subprocess
from pathlib import Path
import boto3
import yaml

def extract_front_matter_and_content_from_markdown(post_path):
    with open(post_path, 'r') as file:
        lines = file.readlines()
        if lines[0].strip() == '---':
            end_line_index = lines[1:].index('---\n') + 1
            front_matter = ''.join(lines[1:end_line_index])
            content = ''.join(lines[end_line_index + 1:])
            return yaml.safe_load(front_matter), content
    return None, None

def get_changed_new_or_deleted_md_files(repo_path):
    # 'git status' 명령어를 실행하여 모든 상태를 얻음
    result = subprocess.run(
        ['git', '-C', repo_path, 'status', '--porcelain=v1'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    
    # 파일 상태와 경로를 포함하는 리스트 생성
    files_status = []
    for line in result.stdout.splitlines():
        # 파일 상태와 경로 분리
        status, file_path = line[:2].strip(), line[3:].strip()
        
        # Markdown 파일만 처리
        if file_path.endswith('.md'):
            file_action = 'modified' if status == 'M' else 'added' if status == 'A' else 'deleted' if status == 'D' else 'untracked'
            files_status.append([file_action, file_path])

    return files_status

# AWS 인증 및 DynamoDB 설정
dynamodb = boto3.resource('dynamodb')

# Posts 테이블 참조
post_table = dynamodb.Table('Posts')
category_table = dynamodb.Table('Categories')

# 깃 저장소의 경로
git_repo_path = '/Users/6mini/post/'
# 변경된, 새로운, 또는 삭제된 파일 목록 가져오기
md_files_status = get_changed_new_or_deleted_md_files(git_repo_path)

if md_files_status:
    commit_slug_list = []
    for status, file_name in md_files_status:
        print([status, file_name])
        file_path = Path(git_repo_path) / file_name
        print(f'Processing file: {file_path}')
        
        post_path = str(file_name).replace(git_repo_path, '')
        main_category, sub_category, slug = post_path.split('/')
        
        main_category = main_category[2:]
        sub_category = sub_category[3:]
        slug = slug[4:].replace('.md', '')
        
        print("main_category:", main_category)
        print("sub_category:", sub_category)
        print("slug:", slug)
        
        if status != 'deleted':
            # 함수를 사용하여 프론트매터와 내용 추출
            front_matter, content = extract_front_matter_and_content_from_markdown(file_path)

            title = front_matter['title']
            description = front_matter['description']
            date = front_matter['date']
            tags = front_matter['tags']

            print(f"title:", title)
            print(f"description:", description)
            print(f"date:", date)
            print(f"tags:", tags)
            
            # 삽입
            response = post_table.put_item(
                Item={
                    'post_id': slug,
                    'title': title,
                    'description': description,
                    'content': content,
                    'date': date,
                    'main_category': main_category,
                    'sub_category': sub_category,
                    'tags': tags
                }
            )

            # 대분류와 관련된 서브 카테고리 가져오기
            response = category_table.get_item(
                Key={
                    'main_category': main_category
                }
            )

            # 대분류가 테이블에 존재하는지 확인
            if 'Item' in response:
                # 해당 대분류에 서브 카테고리가 이미 존재하는지 확인
                if sub_category not in response['Item'].get('sub_categories', []):
                    # 존재하지 않으면 서브 카테고리 추가
                    updated_sub_categories = response['Item']['sub_categories']
                    updated_sub_categories.append(sub_category)
                    category_table.update_item(
                        Key={
                            'main_category': main_category
                        },
                        UpdateExpression="SET sub_categories = :s",
                        ExpressionAttributeValues={
                            ':s': updated_sub_categories
                        }
                    )
            else:
                # 대분류가 존재하지 않으면 새로운 대분류와 서브 카테고리 추가
                category_table.put_item(
                    Item={
                        'main_category': main_category,
                        'sub_categories': [sub_category]
                    }
                )
            
        else:
            # DynamoDB에서 해당 항목 삭제
            response = post_table.delete_item(
                Key={
                    'post_id': slug
                }
            )
            print(f"Deleted post with slug: {slug}")
        
        commit_slug_list.append(f"{status} {slug}")
    if commit_slug_list:
        commit_message = ', '.join(commit_slug_list)
        try:
            # 모든 변경사항 추가
            subprocess.run(f"git add .", check=True, shell=True)

            # 커밋
            subprocess.run(f"git commit -m '{commit_message}'", check=True, shell=True)

            # 푸시
            subprocess.run("git push", check=True, shell=True)
            print("Commit and push successful.")
        except subprocess.CalledProcessError as e:
            print(f"An error occurred: {e}")

['modified', '2-tech/01-web/003-zappa-stage-log-level-management.md']
Processing file: /Users/6mini/post/2-tech/01-web/003-zappa-stage-log-level-management.md
main_category: tech
sub_category: web
slug: zappa-stage-log-level-management
title: 자파(Zappa) 튜토리얼(3): 스테이지(Stage) & 로그 레벨(Log level) 관리
description: 자파(Zappa)와 플라스크(Flask)로 서버리스 애플리케이션을 개발하면서 스테이지 및 로그 레벨 관리의 중요성을 알아본다. 개발, 테스트, 스테이징, 프로덕션 환경을 효율적으로 관리하는 방법과 로그 레벨을 설정하여 비용을 최적화하는 방법에 대한 가이드를 제공한다.
date: 2023-12-04
tags: ['자파(Zappa)', 'AWS', '서버리스(Serverless)', '클라우드(Cloud)', '웹 개발(Web Development)', '스테이지(Stage)', '로그 레벨(Log Level)']
['deleted', '2-tech/03-AWS/004-aws-lambda-eventbridge-scheduling.md']
Processing file: /Users/6mini/post/2-tech/03-AWS/004-aws-lambda-eventbridge-scheduling.md
main_category: tech
sub_category: AWS
slug: aws-lambda-eventbridge-scheduling
Deleted post with slug: aws-lambda-eventbridge-scheduling
['untracked', '2-tech/03-AWS/004-aws-lambda.md']
Processing file: /Users/6mini/post/2-tech/03-AWS/004-aws-

To https://github.com/6mini/post.git
   5c69bec..ff37718  main -> main


In [5]:
import subprocess
from pathlib import Path
import boto3
import yaml

def extract_front_matter_and_content_from_markdown(post_path):
    with open(post_path, 'r') as file:
        lines = file.readlines()
        if lines[0].strip() == '---':
            end_line_index = lines[1:].index('---\n') + 1
            front_matter = ''.join(lines[1:end_line_index])
            content = ''.join(lines[end_line_index + 1:])
            return yaml.safe_load(front_matter), content
    return None, None

# 깃 저장소의 경로
git_repo_path = '/Users/6mini/post/'

def get_changed_or_new_md_files(repo_path):
    # 'git status' 명령어를 실행하여 모든 상태를 얻음
    result = subprocess.run(
        ['git', '-C', repo_path, 'status', '--porcelain=v1'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    
    # 파일 상태와 경로를 포함하는 리스트 생성
    files_status = []
    for line in result.stdout.splitlines():
        # 파일 상태와 경로 분리
        status, file_path = line[:2].strip(), line[3:].strip()
        
        # Markdown 파일만 처리
        if file_path.endswith('.md'):
            if status == '??':  # 새로운 파일
                files_status.append(['new', file_path])
            elif status in ['M', 'A']:  # 수정된 파일 또는 추가된 파일
                files_status.append(['update', file_path])

    return files_status

# 변경된 또는 새로운 파일 목록 가져오기
md_files_status = get_changed_or_new_md_files(git_repo_path)

# 결과 출력
if md_files_status:
    commit_slug_list = []
    for status, file_name in md_files_status:
        print([status, file_name])
        file_path = Path(git_repo_path) / file_name
        print(f'Processing file: {file_path}')
        
        post_path = str(file_name).replace(git_repo_path, '')
            
        main_category, sub_category, slug = post_path.split('/')
        
        main_category = main_category[2:]
        sub_category = sub_category[3:]
        slug = slug[4:].replace('.md', '')
        
        print("main_category:", main_category)
        print("sub_category:", sub_category)
        print("slug:", slug)
        
        # 함수를 사용하여 프론트매터와 내용 추출
        front_matter, content = extract_front_matter_and_content_from_markdown(file_path)

        title = front_matter['title']
        description = front_matter['description']
        date = front_matter['date']
        tags = front_matter['tags']

        print(f"title:", title)
        print(f"description:", description)
        print(f"date:", date)
        print(f"tags:", tags)
        
        # AWS 인증 및 DynamoDB 설정
        dynamodb = boto3.resource('dynamodb')

        # Posts 테이블 참조
        post_table = dynamodb.Table('Posts')
        category_table = dynamodb.Table('Categories')

        # 삽입
        response = post_table.put_item(
            Item={
                'post_id': slug,
                'title': title,
                'description': description,
                'content': content,
                'date': date,
                'main_category': main_category,
                'sub_category': sub_category,
                'tags': tags
            }
        )

        # 대분류와 관련된 서브 카테고리 가져오기
        response = category_table.get_item(
            Key={
                'main_category': main_category
            }
        )

        # 대분류가 테이블에 존재하는지 확인
        if 'Item' in response:
            # 해당 대분류에 서브 카테고리가 이미 존재하는지 확인
            if sub_category not in response['Item'].get('sub_categories', []):
                # 존재하지 않으면 서브 카테고리 추가
                updated_sub_categories = response['Item']['sub_categories']
                updated_sub_categories.append(sub_category)
                category_table.update_item(
                    Key={
                        'main_category': main_category
                    },
                    UpdateExpression="SET sub_categories = :s",
                    ExpressionAttributeValues={
                        ':s': updated_sub_categories
                    }
                )
        else:
            # 대분류가 존재하지 않으면 새로운 대분류와 서브 카테고리 추가
            category_table.put_item(
                Item={
                    'main_category': main_category,
                    'sub_categories': [sub_category]
                }
            )
            
        commit_slug_list.append(f"{status} {slug}")
if commit_slug_list:
    commit_message = ', '.join(commit_slug_list)
    try:
        # 모든 변경사항 추가
        subprocess.run(f"git add .", check=True, shell=True)

        # 커밋
        subprocess.run(f"git commit -m '{commit_message}'", check=True, shell=True)

        # 푸시
        subprocess.run("git push", check=True, shell=True)
        print("Commit and push successful.")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [2]:
commit_slug_list = []
for status, file_name in md_files_status:
    print([status, file_name])
    file_path = Path(git_repo_path) / file_name
    print(f'Processing file: {file_path}')
    
    post_path = str(file_name).replace(git_repo_path, '')
        
    main_category, sub_category, slug = post_path.split('/')
    
    main_category = main_category[2:]
    sub_category = sub_category[3:]
    slug = slug[4:].replace('.md', '')
    
    print("main_category:", main_category)
    print("sub_category:", sub_category)
    print("slug:", slug)
    
    # 함수를 사용하여 프론트매터와 내용 추출
    front_matter, content = extract_front_matter_and_content_from_markdown(file_path)

    title = front_matter['title']
    description = front_matter['description']
    date = front_matter['date']
    tags = front_matter['tags']

    print(f"title:", title)
    print(f"description:", description)
    print(f"date:", date)
    print(f"tags:", tags)
    
    # AWS 인증 및 DynamoDB 설정
    dynamodb = boto3.resource('dynamodb')

    # Posts 테이블 참조
    post_table = dynamodb.Table('Posts')
    category_table = dynamodb.Table('Categories')

    # 삽입
    response = post_table.put_item(
        Item={
            'post_id': slug,
            'title': title,
            'description': description,
            'content': content,
            'date': date,
            'main_category': main_category,
            'sub_category': sub_category,
            'tags': tags
        }
    )

    # 대분류와 관련된 서브 카테고리 가져오기
    response = category_table.get_item(
        Key={
            'main_category': main_category
        }
    )

    # 대분류가 테이블에 존재하는지 확인
    if 'Item' in response:
        # 해당 대분류에 서브 카테고리가 이미 존재하는지 확인
        if sub_category not in response['Item'].get('sub_categories', []):
            # 존재하지 않으면 서브 카테고리 추가
            updated_sub_categories = response['Item']['sub_categories']
            updated_sub_categories.append(sub_category)
            category_table.update_item(
                Key={
                    'main_category': main_category
                },
                UpdateExpression="SET sub_categories = :s",
                ExpressionAttributeValues={
                    ':s': updated_sub_categories
                }
            )
    else:
        # 대분류가 존재하지 않으면 새로운 대분류와 서브 카테고리 추가
        category_table.put_item(
            Item={
                'main_category': main_category,
                'sub_categories': [sub_category]
            }
        )
        
    commit_slug_list.append(f"{status} {slug}")

['update', '2-tech/03-AWS/003-aws-lambda-layer-pandas.md']
Processing file: /Users/6mini/post/2-tech/03-AWS/003-aws-lambda-layer-pandas.md
main_category: tech
sub_category: AWS
slug: aws-lambda-layer-pandas
title: AWS 람다(Lambda) 튜토리얼(3): 계층(Layer) 추가 및 판다스(Pandas) 사용 방법
description: AWS Lambda의 계층(Layer) 기능과 판다스(Pandas) 라이브러리를 Lambda 함수에 통합하는 방법을 자세히 살펴본다. Keith Rozario의 'Klayers' 프로젝트를 활용하여 판다스를 쉽게 사용하는 방법과 Lambda에서의 계층 추가 및 관리 방법을 배운다. 이를 통해 Python을 사용한 데이터 처리와 분석을 효율적으로 수행할 수 있는 방법을 탐구한다. 또한, Lambda의 서버리스 아키텍처를 통한 코드 재사용성과 라이브러리 관리의 효율성을 극대화하는 방법에 대해서도 알아볼 것이다.
date: 2024-01-17
tags: ['AWS', '람다(Lambda)', '서버리스(Serverless)', '클라우드(Cloud)', '튜토리얼(Tutorial)', '계층(Layer)', '판다스(Pandas)']
['new', '2-tech/03-AWS/004-aws-lambda-eventbridge-scheduling.md']
Processing file: /Users/6mini/post/2-tech/03-AWS/004-aws-lambda-eventbridge-scheduling.md
main_category: tech
sub_category: AWS
slug: aws-lambda-eventbridge-scheduling
title: AWS Lambda(Lambda) 튜토리얼(4): 이벤트 브릿지(EventBridge)를 통한 스케쥴링
desc

In [3]:
commit_message = ', '.join(commit_slug_list)
commit_message

'update aws-lambda-layer-pandas, new aws-lambda-eventbridge-scheduling'

In [4]:
try:
    # 모든 변경사항 추가
    subprocess.run(f"git add .", check=True, shell=True)

    # 커밋
    subprocess.run(f"git commit -m '{commit_message}'", check=True, shell=True)

    # 푸시
    subprocess.run("git push", check=True, shell=True)
    print("Commit and push successful.")
except subprocess.CalledProcessError as e:
    print(f"An error occurred: {e}")

[main 5c69bec] update aws-lambda-layer-pandas, new aws-lambda-eventbridge-scheduling
 3 files changed, 240 insertions(+), 19 deletions(-)
 create mode 100644 2-tech/03-AWS/004-aws-lambda-eventbridge-scheduling.md
Commit and push successful.


To https://github.com/6mini/post.git
   1e0cca1..5c69bec  main -> main


In [27]:
# 최초

from pathlib import Path
import boto3
import yaml

def extract_front_matter_and_content_from_markdown(post_path):
    with open(post_path, 'r') as file:
        lines = file.readlines()
        if lines[0].strip() == '---':
            end_line_index = lines[1:].index('---\n') + 1
            front_matter = ''.join(lines[1:end_line_index])
            content = ''.join(lines[end_line_index + 1:])
            return yaml.safe_load(front_matter), content
    return None, None

path = '/Users/6mini/post/'

for category in ['1-life/', '2-tech/']:
    root_directory = Path(path + category)

    # .md 파일을 재귀적으로 모두 찾기
    md_files = root_directory.glob('**/*.md')

    # 각 Markdown 파일에 대해 처리
    for md_file in md_files:
        # 파일의 전체 경로와 폴더 이름을 출력 (예시)
        print(f'Processing file: {md_file}')
        post_path = str(md_file).replace(path, '')
        
        main_category, sub_category, slug = post_path.split('/')
        
        main_category = main_category[2:]
        sub_category = sub_category[3:]
        slug = slug[4:].replace('.md', '')
        
        print("main_category:", main_category)
        print("sub_category:", sub_category)
        print("slug:", slug)
        
        # 함수를 사용하여 프론트매터와 내용 추출
        front_matter, content = extract_front_matter_and_content_from_markdown(md_file)

        title = front_matter['title']
        description = front_matter['description']
        date = front_matter['date']
        tags = front_matter['tags']

        print(f"title:", title)
        print(f"description:", description)
        print(f"date:", date)
        print(f"tags:", tags)
        
        # AWS 인증 및 DynamoDB 설정
        dynamodb = boto3.resource('dynamodb')

        # Posts 테이블 참조
        post_table = dynamodb.Table('Posts')
        category_table = dynamodb.Table('Categories')

        # 삽입
        response = post_table.put_item(
            Item={
                'post_id': slug,
                'title': title,
                'description': description,
                'content': content,
                'date': date,
                'main_category': main_category,
                'sub_category': sub_category,
                'tags': tags
            }
        )

        # 대분류와 관련된 서브 카테고리 가져오기
        response = category_table.get_item(
            Key={
                'main_category': main_category
            }
        )

        # 대분류가 테이블에 존재하는지 확인
        if 'Item' in response:
            # 해당 대분류에 서브 카테고리가 이미 존재하는지 확인
            if sub_category not in response['Item'].get('sub_categories', []):
                # 존재하지 않으면 서브 카테고리 추가
                updated_sub_categories = response['Item']['sub_categories']
                updated_sub_categories.append(sub_category)
                category_table.update_item(
                    Key={
                        'main_category': main_category
                    },
                    UpdateExpression="SET sub_categories = :s",
                    ExpressionAttributeValues={
                        ':s': updated_sub_categories
                    }
                )
        else:
            # 대분류가 존재하지 않으면 새로운 대분류와 서브 카테고리 추가
            category_table.put_item(
                Item={
                    'main_category': main_category,
                    'sub_categories': [sub_category]
                }
            )

Processing file: /Users/6mini/post/1-life/01-reflections/002-2021-retrospect.md
main_category: life
sub_category: reflections
slug: 2021-retrospect
title: 2021년 회고
description: 꿈꾸던 데이터 엔지니어가 되기 위해 노력한 이윤민의 좌충우돌 2021년 한 해를 돌아본다.
date: 2022-01-06
tags: ['회고(Retrospect)']
Processing file: /Users/6mini/post/1-life/01-reflections/004-personal-blog-development-open.md
main_category: life
sub_category: reflections
slug: personal-blog-development-open
title: 개인 블로그 개발 및 오픈: 새로운 목표의 시작
description: 여러 블로그 플랫폼을 경험한 후, 왜 개인 블로그 개발을 결심하게 되었는지에 대한 이야기를 나눈다. 블로그를 통해 표현하고자 하는 나만의 가치와 메시지, 그리고 이를 실현하기 위해 선택한 기술적 도전과 개인 브랜드 구축에 대해 이야기한다. 'LIFE'와 'TECH' 두 가지 카테고리를 중심으로, 개인적 계발과 기술적 성장을 추구하는 이유와 계획을 소개한다.
date: 2023-12-11
tags: ['회고(Retrospect)', '웹 개발(Web Development)', '블로그 개발(Blog Development)', '퍼스널 브랜딩(Personal Branding)']
Processing file: /Users/6mini/post/1-life/01-reflections/006-2023-retrospect.md
main_category: life
sub_category: reflections
slug: 2023-retrospect
title: 2023년 회고
description: 20

In [ ]:
if COMMIT:
    import subprocess

    if NEW:
        msg = 'post'
    else:
        msg = 'update'
        
    try:
        # 모든 변경사항 추가
        subprocess.run(f"git add '{post_path}'", check=True, shell=True)

        # 커밋
        subprocess.run(f"git commit -m '{msg}: {title}'", check=True, shell=True)

        # 푸시
        subprocess.run("git push", check=True, shell=True)
        print("Commit and push successful.")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [3]:
import os

COMMIT = 1
NEW = 1
# md 경로 복사
post_path = '1-life/01-reflections/001-codestates-ai-bootcamp-review/codestates-ai-bootcamp-review.md'

main_category, sub_category, slug, md_file = post_path.split('/')

main_category = main_category[2:]
sub_category = sub_category[3:]
slug = slug[4:]

commit_path = post_path.replace('/' + md_file, '')

main_category, sub_category, slug, md_file, commit_path

('tech',
 'AWS',
 'aws-lambda-layer-pandas',
 'aws-lambda-layer-pandas.md',
 '2-tech/03-AWS/003-aws-lambda-layer-pandas')

In [4]:
# 마크다운 파일 가져오기
import yaml

def extract_front_matter_and_content_from_markdown(post_path):
    with open(post_path, 'r') as file:
        lines = file.readlines()
        if lines[0].strip() == '---':
            end_line_index = lines[1:].index('---\n') + 1
            front_matter = ''.join(lines[1:end_line_index])
            content = ''.join(lines[end_line_index + 1:])
            return yaml.safe_load(front_matter), content
    return None, None

# 함수를 사용하여 프론트매터와 내용 추출
front_matter, content = extract_front_matter_and_content_from_markdown(post_path)

title = front_matter['title']
description = front_matter['description']
date = front_matter['date']
tags = front_matter['tags']

print(f"title: {title}")
print(f"description: {description}")
print(f"date: {date}")
print(f"tags: {tags}")



# AWS 인증 및 DynamoDB 설정
dynamodb = boto3.resource('dynamodb')

# Posts 테이블 참조
post_table = dynamodb.Table('Posts')
category_table = dynamodb.Table('Categories')

# 삽입
response = post_table.put_item(
    Item={
        'post_id': slug,
        'title': title,
        'description': description,
        'content': content,
        'date': date,
        'main_category': main_category,
        'sub_category': sub_category,
        'tags': tags
    }
)

# 대분류와 관련된 서브 카테고리 가져오기
response = category_table.get_item(
    Key={
        'main_category': main_category
    }
)

# 대분류가 테이블에 존재하는지 확인
if 'Item' in response:
    # 해당 대분류에 서브 카테고리가 이미 존재하는지 확인
    if sub_category not in response['Item'].get('sub_categories', []):
        # 존재하지 않으면 서브 카테고리 추가
        updated_sub_categories = response['Item']['sub_categories']
        updated_sub_categories.append(sub_category)
        category_table.update_item(
            Key={
                'main_category': main_category
            },
            UpdateExpression="SET sub_categories = :s",
            ExpressionAttributeValues={
                ':s': updated_sub_categories
            }
        )
else:
    # 대분류가 존재하지 않으면 새로운 대분류와 서브 카테고리 추가
    category_table.put_item(
        Item={
            'main_category': main_category,
            'sub_categories': [sub_category]
        }
    )



title: AWS 람다(Lambda) 튜토리얼(3): 계층(Layer) 추가 및 판다스(Pandas) 사용 방법
description: AWS Lambda의 계층(Layer) 기능과 판다스(Pandas) 라이브러리를 Lambda 함수에 통합하는 방법을 자세히 살펴본다. Keith Rozario의 'Klayers' 프로젝트를 활용하여 판다스를 쉽게 사용하는 방법과 Lambda에서의 계층 추가 및 관리 방법을 배운다. 이를 통해 Python을 사용한 데이터 처리와 분석을 효율적으로 수행할 수 있는 방법을 탐구한다. 또한, Lambda의 서버리스 아키텍처를 통한 코드 재사용성과 라이브러리 관리의 효율성을 극대화하는 방법에 대해서도 알아볼 것이다.
date: 2024-01-17
tags: ['AWS', '람다(Lambda)', '서버리스(Serverless)', '클라우드(Cloud)', '튜토리얼(Tutorial)', '계층(Layer)', '판다스(Pandas)']
[main dfacd6e] post: AWS 람다(Lambda) 튜토리얼(3): 계층(Layer) 추가 및 판다스(Pandas) 사용 방법
 1 file changed, 163 insertions(+)
 create mode 100644 2-tech/03-AWS/003-aws-lambda-layer-pandas/aws-lambda-layer-pandas.md


KeyboardInterrupt: 

remote: error: Trace: 1d737ff463138edc45bf59c125c980525cd5c2323d4b22902cb328f04d2b4ef5        
remote: error: See https://gh.io/lfs for more information.        
remote: error: File 1-life/01-reflections/006-2023-retrospect/2023-retrospect-3.gif is 104.74 MB; this exceeds GitHub's file size limit of 100.00 MB        
remote: error: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.        
